In [1]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2161
Receiving objects: 100% (2166/2166), 176.31 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

In [3]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
def load_house_attributes(inputPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=' ', header=None, names=cols)
    
    zipcodes = df['zipcode'].value_counts().keys().tolist()
    counts   = df['zipcode'].value_counts().tolist()
    #df.head()
    
    for (zipcode, count) in zip(zipcodes, counts):
      
        if count < 25:
            idxs = df[df['zipcode'] == zipcode].index
            df.drop(idxs, inplace=True)
            
    return df
  
def process_house_attributes(df, train, test):
    continuous = ['bedrooms', 'bathrooms', 'area']
    
    cs               = MinMaxScaler()
    train_continuous = cs.fit_transform(train[continuous])
    test_continuous  = cs.transform(test[continuous])
    
    zip_binarizer     = LabelBinarizer().fit(df['zipcode'])
    train_categorical = zip_binarizer.transform(train['zipcode'])
    test_categorical  = zip_binarizer.transform(test['zipcode'])
    
    train_X = np.hstack([train_categorical, train_continuous])
    test_X  = np.hstack([test_categorical, test_continuous])
    
    return (train_X, test_X)
  
def load_house_images(df, inputPath):
    images = []
    
    for i in df.index.values:
        base_path   = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        house_paths = sorted(list(glob.glob(base_path)))
        
        input_images = []
        output_image = np.zeros((64, 64, 3), dtype="uint8")
        
        for house_path in house_paths:
            image = cv2.imread(house_path)
            image = cv2.resize(image, (32, 32))
            input_images.append(image)
            
        output_image[0:32, 0:32]   = input_images[0]
        output_image[0:32, 32:64]  = input_images[1]
        output_image[32:64, 32:64] = input_images[2]
        output_image[32:64, 0:32]  = input_images[3]
        
        images.append(output_image)
        
    return np.array(images)
        

In [0]:
def create_mlp(dim, regress=False):
    # define our MLP network
	  model = Sequential()
	  model.add(Dense(8, input_dim=dim, activation="relu"))
	  model.add(Dense(4, activation="relu"))
 
	  # check to see if the regression node should be added
	  if regress:
		    model.add(Dense(1, activation="linear"))
 
	  # return our model
	  return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
  
    input_shape = (height, width, depth)
    chan_dim = -1
    
    inputs = Input(shape=input_shape)
    
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chan_dim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chan_dim)(x)
    x = Dropout(0.5)(x)
    
    x = Dense(4)(x)
    x = Activation("relu")(x)
    
    if regress:
        x = Dense(1, activation="linear")(x)
        
    model = Model(inputs, x)
    
    return model

In [0]:
cwd = os.getcwd()
house_jpg_dir = os.path.join(cwd, 'Houses-dataset', 'Houses Dataset')
houses_info   = os.path.join(house_jpg_dir, 'HousesInfo.txt')

df = load_house_attributes(houses_info)

images = load_house_images(df, house_jpg_dir)

# Scaling pixel intensities to the range [0, 1].
images = images / 255.0

In [0]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [0]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [0]:
(trainAttrX, testAttrX) = process_house_attributes(df, trainAttrX, testAttrX)

In [10]:
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(64, 64, 3, regress=False)

combinedInput = concatenate([mlp.output, cnn.output])
 
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
 
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

W0730 13:41:29.776602 140380935870336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 13:41:29.817975 140380935870336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 13:41:29.825686 140380935870336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0730 13:41:29.914289 140380935870336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0730 13:41:29.915540 1403809358

In [11]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
model.summary()

W0730 13:41:30.664004 140380935870336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          activation_1[0][0]               
__________________________________________________________________________________________________
max_poolin

In [12]:
model.fit([trainAttrX, trainImagesX], trainY, 
          validation_data=([testAttrX, testImagesX], testY),	
          epochs=200, 
          batch_size=8)

model.save("housing_prices_stats_and_images_model.h5")

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 4s 15ms/step - loss: 556.2186 - val_loss: 1337.9927
Epoch 2/200
271/271 [==============================] - 2s 9ms/step - loss: 421.8579 - val_loss: 802.7460
Epoch 3/200
271/271 [==============================] - 2s 9ms/step - loss: 277.8333 - val_loss: 408.6861
Epoch 4/200
271/271 [==============================] - 2s 9ms/step - loss: 186.4672 - val_loss: 130.0478
Epoch 5/200
271/271 [==============================] - 3s 9ms/step - loss: 126.8539 - val_loss: 99.3998
Epoch 6/200
271/271 [==============================] - 2s 9ms/step - loss: 102.3180 - val_loss: 76.5035
Epoch 7/200
271/271 [==============================] - 3s 9ms/step - loss: 111.1502 - val_loss: 72.6740
Epoch 8/200
271/271 [==============================] - 3s 9ms/step - loss: 80.0381 - val_loss: 65.4107
Epoch 9/200
271/271 [==============================] - 2s 9ms/step - loss: 91.2461 - val_loss: 66.8556
Epoch 10/200
27

In [0]:
preds = model.predict([testAttrX, testImagesX])

In [37]:
import locale

diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. house price: $533,388.27, std house price: $493,403.08
mean: 22.17%, std: 19.80%


In [0]:
from google.colab import files

files.download("housing_prices_stats_and_images_model.h5")